In [1]:
import pprint
import os
import json
from SPARQLWrapper import SPARQLWrapper, JSON
import boto3
import pandas as pd
import re

In [3]:
###############Query###########################
query = """PREFIX lm: <http://ontology.lungmap.net/ontologies/expression_ontology#>
PREFIX mont: <http://ontology.lungmap.net/ontologies/mouse_anatomy#>
PREFIX owl: <http://www.w3.org/2002/07/>
PREFIX owl2: <http://www.w3.org/2002/07/owl#>
SELECT DISTINCT ?experiment_id ?image_id 
?path ?raw_file
?experiment_type_label ?experiment_type_id
?term ?term_label ?experiment
FROM  <http://data.lungmap.net/lungmap_data>
FROM  <http://data.lungmap.net/lungmap_ontology>
FROM  <http://data.lungmap.net/lungmap_ontology>

WHERE {
   ?experiment_id lm:is_experiment_type/rdfs:label ?experiment_type_label .
   ?image_id lm:part_of_experiment ?experiment_id .
   ?experiment_id lm:file_name ?path .
   ?image_id lm:display_url ?raw_file .
   ?experiment_id lm:is_experiment_type ?experiment_type_id .

   ?experiment_id lm:has_condition ?term .
   ?term rdfs:label ?term_label .

}
"""

In [4]:
pp = pprint.PrettyPrinter(indent=2)

sparql = SPARQLWrapper("http://testdata.lungmap.net/sparql")
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
#Here are all the results form the sparql query above
results = sparql.query().convert()

##############################################################################
############################################################################## 
##############################################################################
#len(results['results']['bindings'])
index = []
#only want to do .tif files (check with Cliburn), so get index of those files
for i,x in enumerate(results['results']['bindings']):
    filename = x.get('raw_file').get('value')
    name, ext = os.path.splitext(filename)
    if (ext=='.tif' or ext=='.tiff'):
        #print(ext)
        index.append(i)

In [5]:
tif_files = [results['results']['bindings'][i] for i in index]
#Define a client connection to our bucket for download
s3 = boto3.resource('s3')
bucket = s3.Bucket('lungmap-breath-data')

In [8]:
regexp = re.compile("#(.*)$")
def bens_dict_2_flatfile(metadata_from_sparql):
    #Here are the "variables that I want in my flat file:
    s3downloadkey = [] #I can parse this to get file name if needed and this should be unique by metadata object (primary key)
    image_id = []
    experiment_id = []
    term = []
    experiment = []
    term_label = []
    
    for i, x in enumerate(metadata_from_sparql):
        #first, I'll create my s3key name
        filename = x.get('raw_file').get('value')
        name, ext = os.path.splitext(filename)
        root = os.path.basename(os.path.normpath(x.get('path').get('value')))
        s3objkey =  os.path.join(root,name,filename)
        #now let's start appending to our lists
        s3downloadkey.append(s3objkey)
        
        tm = regexp.search(os.path.basename(os.path.normpath(x.get('term').get('value')))).group(1)
        term.append(tm)
        exp_id = regexp.search(os.path.basename(os.path.normpath(x.get('experiment_type_id').get('value')))).group(1)
        experiment_id.append(exp_id)
        term_label.append(x.get('term_label').get('value'))
        im_id = regexp.search(os.path.normpath(x.get('image_id').get('value'))).group(1)
        image_id.append(im_id)
        output = pd.DataFrame({  #'s3downloadkey' : s3downloadkey,
                             'experiment_id': experiment_id,
                             'term_label': term_label,
                             'term': term,
                             'image_id': image_id
                              },index=s3downloadkey)
    return(output)

In [9]:
flat_file = bens_dict_2_flatfile(tif_files[:]) 
#note that we now have the key to download from boto3
#i.e. bucket.download_file(s3objkey, os.path.join(filepath, filename)) 
#os.chdir('/Users/nn31/Dropbox/40-githubRrepos/lungmap-data-plus-2016/data') 
flat_file.to_csv('C:/Users/Lina/Desktop/metadata/metadata_dictionary_annotation_terms.csv',index=False)    #this file ends up in os.getcwd()               

In [7]:
our_images = pd.read_csv('C:/Users/Lina/Desktop/metadata/00_metadata_dictionary.csv')  
new_output = pd.read_csv('C:/Users/Lina/Desktop/metadata/metadata_dictionary_annotation_terms.csv').loc[pd.read_csv('C:/Users/Lina/Desktop/metadata/metadata_dictionary_annotation_terms.csv')['image_id'].isin(our_images.image_id.tolist())]
new_output.shape==our_images.shape                   
#uh oh, what's going on?
list(set(our_images.image_id.tolist()) - set(new_output.image_id.tolist()))


['LMEX0000000676_IMG_1',
 'LMEX0000000675_IMG_4',
 'LMEX0000000675_IMG_3',
 'LMEX0000000675_IMG_2',
 'LMEX0000000675_IMG_5',
 'LMEX0000000676_IMG_3',
 'LMEX0000000675_IMG_6',
 'LMEX0000000676_IMG_4',
 'LMEX0000000676_IMG_5',
 'LMEX0000000676_IMG_2',
 'LMEX0000000675_IMG_1']